In [1]:
%load_ext autoreload
%autoreload 2

In [48]:
import pathlib
import numpy as np
from src.visualization import visualize_pointcloud,visualize_pointclouds_parts, visualize_mesh
import trimesh

root = pathlib.Path("/mnt/hdd/shapenetcore_partanno_segmentation_benchmark_v0/03001627/")
shapenet_root = pathlib.Path("/mnt/hdd/ShapeNetCore.v2/03001627/")


In [49]:
root

PosixPath('/mnt/hdd/shapenetcore_partanno_segmentation_benchmark_v0/03001627')

In [71]:
n_max = 5
points = []
labels = []
meshes = []

for i, point_path in enumerate(root.joinpath("points").iterdir()):
    if i == n_max:
        break
    
    points.append(np.loadtxt(point_path))
    labels.append(np.loadtxt(root.joinpath("points_label").joinpath(point_path.stem).with_suffix(".seg"), dtype=int))

    with open(shapenet_root.joinpath(point_path.stem).joinpath("models").joinpath("model_normalized.obj")) as fp:
        meshes.append(trimesh.exchange.obj.load_obj(fp, include_color=False, include_texture=False))

WARNING - 2022-11-03 01:44:16,130 - obj - specified material (material_1_24)  not loaded!
WARNING - 2022-11-03 01:44:16,139 - obj - specified material (material_2_24)  not loaded!
WARNING - 2022-11-03 01:44:16,167 - obj - specified material (material_1_24)  not loaded!
WARNING - 2022-11-03 01:44:28,335 - obj - specified material (material_0_24)  not loaded!
WARNING - 2022-11-03 01:44:28,363 - obj - specified material (material_0_24)  not loaded!
WARNING - 2022-11-03 01:44:28,374 - obj - specified material (material_0_24)  not loaded!


In [72]:
visualize_pointclouds_parts(points, labels)

Output()

In [77]:
for i in range(5):
    print(meshes[i].keys())

dict_keys(['geometry', 'graph'])
dict_keys(['geometry', 'graph'])
dict_keys(['include_color', 'include_texture', 'faces', 'vertices', 'vertex_normals', 'visual'])
dict_keys(['include_color', 'include_texture', 'faces', 'vertices', 'vertex_normals', 'visual'])
dict_keys(['geometry', 'graph'])


In [79]:
meshes[0]["graph"]

[{'geometry': 'material_1_24',
  'frame_to': 'material_1_24',
  'matrix': array([[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]])},
 {'geometry': 'material_2_24',
  'frame_to': 'material_2_24',
  'matrix': array([[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]])},
 {'geometry': 'material_0_1_8',
  'frame_to': 'material_0_1_8',
  'matrix': array([[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]])}]

In [66]:
visualize_mesh(meshes[0]["geometry"]["material_1_24"]["vertices"], meshes[0]["geometry"]["material_1_24"]["faces"])
visualize_mesh(meshes[0]["geometry"]["material_2_24"]["vertices"], meshes[0]["geometry"]["material_2_24"]["faces"])
visualize_mesh(meshes[0]["geometry"]["material_0_1_8"]["vertices"], meshes[0]["geometry"]["material_0_1_8"]["faces"])

Output()

Output()

Output()

In [45]:
point_path

PosixPath('/mnt/hdd/shapenetcore_partanno_segmentation_benchmark_v0/03001627/points/e8dc06c3d01d6592f4b6538438a0b930.pts')

In [6]:
import open3d as o3d
from open3d import JVisualizer

INFO - 2022-11-03 00:56:10,423 - utils - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2022-11-03 00:56:10,423 - utils - NumExpr defaulting to 8 threads.


In [22]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[0])

In [23]:
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(100)

In [29]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=12)

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 2626 / 2626
[Open3D DEBUG] #   Got kernel density: 0.0595992 (s), 307.324 (MB) / 307.324 (MB) / 307 (MB)
[Open3D DEBUG] #     Got normal field: 0.0156589 (s), 320.203 (MB) / 320.203 (MB) / 320 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 4.473238e-04 / 1.174672e+00
[Open3D DEBUG] #       Finalized tree: 0.0726418 (s), 339.023 (MB) / 339.023 (MB) / 339 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.0648761 (s), 324.199 (MB) / 339.023 (MB) / 339 (MB)
[Open3D DEBUG] #Set point constraints: 0.00894594 (s), 324.199 (MB) / 339.023 (MB) / 339 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 569388 / 88032 / 562697
[Open3D DEBUG] Memory Usage: 324.199 MB
Cycle[0] Depth[ 0/12]:	Updated constraints / Got system / Solved in:  0.000 /  0.000 /  0.000	(324.199 MB)	Nodes: 8
CG: 4.9575e-01 -> 4.9575e-01 -> 3.3031e-04 (6.7e-04) [0]
Cycle[0] Depth[ 1/12]:	Updated constraints / Got system / Solved in:  0.001 /  0.

In [36]:
alpha=0.15
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)

In [41]:
radii = [0.005, 0.01, 0.02, 0.04, 0.08]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))

In [42]:
visualize_mesh(np.asarray(mesh.vertices), np.asarray(mesh.triangles))

Output()